In [1]:
%%time
%pip install catboost
#%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
#%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
%pip install polars
%pip install tqdm
#%pip install wandb
#%pip install sweetviz
#%pip install --upgrade scipy
#%pip install -q -U autogluon.tabular
#%pip install autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 171.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 208.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 182.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 125.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 178.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 2.71 s, sys: 1.12 s, total: 3.83 s
Wall time: 44.9 s


In [2]:

import numpy as np
from numpy import random
#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
#import matplotlib.pyplot as plt
#from fastai.imports import *
#np.set_printoptions(linewidth=130)
from pathlib import Path
import os
import xgboost as xgb
#from xgboost import plot_importance
from xgboost import XGBRegressor
import warnings
import gc
import pickle
from joblib import dump, load
import typing as t
#import bentoml
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_val_score,train_test_split
#import wandb

torch.manual_seed(42)

In [6]:
path = Path('')

In [7]:
!ls

pharma_sales.csv  pharma_sales.ipynb


In [8]:
df = pd.read_csv(path/'pharma_sales.csv')

In [9]:
df

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product Name,Product Class,Quantity,Price,Sales,Month,Year,Name of Sales Rep,Manager,Sales Team
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254077,Bashirian-Kassulke,"Koch, Borer and Hagenes Pharmaceutical Ltd",Lauf,Germany,49.5103,11.2772,Hospital,Private,Pentastrin,Antibiotics,919.0,497,456743.0,December,2020,Thompson Crawford,James Goodwill,Alfa
254078,Bashirian-Kassulke,Hane Ltd Pharmaceutical Ltd,Aichach,Germany,48.4500,11.1333,Hospital,Private,Abranatal Lysoprosate,Antiseptics,432.0,681,294192.0,December,2020,Anne Wu,Britanny Bold,Delta
254079,Bashirian-Kassulke,Harris-Conroy Pharmacy,Wilhelmshaven,Germany,53.5167,8.1333,Pharmacy,Retail,Adideine,Mood Stabilizers,320.0,678,216960.0,December,2020,Abigail Thompson,Tracy Banks,Bravo
254080,Bashirian-Kassulke,Balistreri Group Pharm,Böblingen,Germany,48.6833,9.0000,Hospital,Government,Feruprazole,Mood Stabilizers,565.0,115,64975.0,December,2020,Stella Given,Alisha Cordwell,Charlie


In [11]:
# Get the minimum and maximum dates
min_date = df['Year'].min()
max_date = df['Year'].max()

print(f"Date range: {min_date} to {max_date}")

# For more detailed information about the date distribution
print("\nDate distribution summary:")
print(df['Year'].describe())

Date range: 2017 to 2020

Date distribution summary:
count    254082.000000
mean       2018.385187
std           1.041352
min        2017.000000
25%        2018.000000
50%        2018.000000
75%        2019.000000
max        2020.000000
Name: Year, dtype: float64


In [26]:
# Split based on Year column only
train_df = df[df['Year'] < 2020]
test = df[df['Year'] == 2020]

# Verify the split
print("Training set:")
print("Years covered:", sorted(train_df['Year'].unique()))
print("Months covered:", sorted(train_df['Month'].unique()))
print("Size:", len(train_df))

print("\nTest set:")
print("Years covered:", sorted(test['Year'].unique()))
print("Months covered:", sorted(test['Month'].unique()))
print("Size:", len(test_df))

Training set:
Years covered: [2017, 2018, 2019]
Months covered: ['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']
Size: 202889

Test set:
Years covered: [2020]
Months covered: ['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']
Size: 51193


In [27]:
test_df = test.drop('Sales', axis=1)

In [28]:
#train_df = pd.read_csv(path/'train.csv')
#test_df = pd.read_csv(path/'test.csv')
#sub_df = pd.read_csv(path/'sample_submission.csv')
missing_values_count = train_df.isnull().sum()
sorted_missing_values = missing_values_count.sort_values(ascending=False)
print(sorted_missing_values)

Distributor          0
Customer Name        0
City                 0
Country              0
Latitude             0
Longitude            0
Channel              0
Sub-channel          0
Product Name         0
Product Class        0
Quantity             0
Price                0
Sales                0
Month                0
Year                 0
Name of Sales Rep    0
Manager              0
Sales Team           0
dtype: int64


In [29]:
missing_values_count = test_df.isnull().sum()
sorted_missing_values = missing_values_count.sort_values(ascending=False)
print(sorted_missing_values)

Distributor          0
Customer Name        0
City                 0
Country              0
Latitude             0
Longitude            0
Channel              0
Sub-channel          0
Product Name         0
Product Class        0
Quantity             0
Price                0
Month                0
Year                 0
Name of Sales Rep    0
Manager              0
Sales Team           0
dtype: int64


In [30]:
# Save training data
train_df.to_csv('pharmaceutical_sales_train.csv', index=False)

# Save test data
test_df.to_csv('pharmaceutical_sales_test.csv', index=False)

In [31]:
#train_df = add_datepart(train_df,'Year',drop=False)
#test_df = add_datepart(test_df,'Year',drop=False)

In [32]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='Sales')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
#to = TabularPandas(train_df, procs=[Categorify,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Sales',
                   y_block=CategoryBlock(),
                   splits=splits)
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test_df)

In [33]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [34]:
xgb_model = xgb.XGBRegressor()
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

xgb_score = mean_absolute_percentage_error(y_test,xgb_preds_x)
print(f"Model MAPE: {xgb_score}")


Model MAPE: 442.89158855223803


In [35]:
xgb_preds

tensor([  976.2667,  7652.2729,  8832.7764,  ..., 12452.5732,  9027.4805,
        11370.1738])